In [1]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

feature_file = '../preprocess/MSRII-c3d-features.h5'

with h5py.File(feature_file, 'r') as h5:
    X = np.array(h5['features'])
    Y = np.array(h5['labels']).reshape(X.shape[0])
    Y[Y > 0] = 1

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [2]:
%env KERAS_BACKEND=theano
from keras.models import Sequential
from keras.layers import Dense, Dropout

import model as m


def FC1Net():
    model = Sequential()
    model.add(Dense(1, input_dim=4096, activation='sigmoid'))
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


def FC4Net():
    net = m.FC4Net()
    net.compile_model()
    return net.model

model_creator = FC4Net
model = model_creator()
H = model.fit(
    X_train, y_train,
    nb_epoch=10, batch_size=32,
    validation_data=(X_test, y_test), verbose=0)

env: KERAS_BACKEND=theano


Using Theano backend.
Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)


In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

estimator = KerasClassifier(build_fn=model_creator, nb_epoch=10, batch_size=32, verbose=0)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)

print("Results: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=model_creator, nb_epoch=10, batch_size=32, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)

print("Results: %.2f%% (%.2f%%) Feature normalized" % (results.mean() * 100, results.std() * 100))

Results: 89.35% (1.47%)
Results: 89.51% (2.51%) Feature normalized


In [4]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32, verbose=0)
print('Results: %.2f%%' % (accuracy * 100))

Results: 91.74%


In [5]:
model.save_weights('FC4Net_weights.h5')

In [6]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

clf = svm.SVC(kernel='rbf', C=1).fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print('SVM accuracy %.2f%%' % (accuracy * 100))

clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print('RF accuracy %.2f%%' % (accuracy * 100))

SVM accuracy 92.80%
RF accuracy 91.95%
